In [8]:
import cv2
import mediapipe as mp



In [9]:

import os

model_path = 'D:/proyectos ML/VisionArtificial/Mascarillas dataset/mask_model.xml'  # Ruta del modelo

if os.path.exists(model_path):
    print(f"El archivo {model_path} existe.")
else:
    print(f"Error: El archivo {model_path} no se encuentra en la ruta especificada.")


El archivo D:/proyectos ML/VisionArtificial/Mascarillas dataset/mask_model.xml existe.


In [10]:
mp_face_detection = mp.solutions.face_detection
LABELS = ['Con_mascarilla', 'Sin_mascarilla']

face_mask = cv2.face.LBPHFaceRecognizer_create()
face_mask.read(model_path)

In [11]:
cap = cv2.VideoCapture(0)

# Inicializa Mediapipe FaceDetection
mp_face_detection = mp.solutions.face_detection
with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Voltea la imagen horizontalmente
        frame = cv2.flip(frame, 1)
        high, width, _ = frame.shape

        # Convierte el frame a RGB para mediapipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame_rgb)

        # Si se detectaron caras
        if results.detections is not None:
            for detection in results.detections:
                # Calcula las coordenadas de la caja delimitadora
                xmin = int(detection.location_data.relative_bounding_box.xmin * width)
                ymin = int(detection.location_data.relative_bounding_box.ymin * high)
                w = int(detection.location_data.relative_bounding_box.width * width)
                h = int(detection.location_data.relative_bounding_box.height * high)
                
                # Si las coordenadas están fuera del frame, omitir
                if xmin < 0 or ymin < 0:
                    continue
                
                
                face_img = frame[max(0, ymin):min(high, ymin+h), max(0, xmin):min(width, xmin+w)]
                face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
                face_img  = cv2.resize(face_img, (72, 72), interpolation=cv2.INTER_CUBIC)
                # Muestra la cara recortada
                cv2.imshow('Face', face_img)
                
                results = face_mask.predict(face_img)
                cv2.putText(frame, "{}".format(results), (xmin, ymin-5), 1, 1.3, (210,124,176), 1, cv2.LINE_AA)
                #cv2.rectangle(frame, (xmin, ymin), (xmin+w, ymin+h), (255, 0, 0), 2)
                
                # Si el resultado de la predicción es menor que el umbral de 138
                if results[1] < 138:
                    # Determina el color en función de si tiene o no mascarilla
                    color = (0, 255, 0) if LABELS[results[0]] == 'Con_mascarilla' else (0, 0, 255)
                    cv2.putText(frame, "{}".format(LABELS[results[0]]), (xmin, ymin-25), 2, 1, color, 1, cv2.LINE_AA)
                    cv2.rectangle(frame, (xmin, ymin), (xmin+w, ymin+h), color, 2)

                    
        
        # Muestra el frame completo
        cv2.imshow('Frame', frame)
        
        # Espera por la tecla 'ESC' para cerrar la ventana
        k = cv2.waitKey(1) 
        if k == 27:  # Si se presiona la tecla 'ESC'
            break

# Libera la cámara y destruye las ventanas
cap.release()
cv2.destroyAllWindows()